# Import the Data and Wrangle to focus specifically on Bats

In [48]:
#Import necessary objects from other notebooks
%store -r data2025_dir site_dir


In [94]:
# Import necessary packages

# Importing and accessing CSC
import pandas as pd
import os
import re

# Working with Dataframes
import geopandas as gpd
from shapely.geometry import Point

In [77]:
def load_csv_data(csv_filename, base_dir, header='infer', names=None):
    """
    Loads a CSV file from a given base directory with a flexible header and column name option.
    
    Parameters:
        csv_filename (str): Name of the CSV file (e.g., 'fire_stats.csv')
        base_dir (str): Directory path where the file lives
        header (int, str, or None): Row to use as column names, or None if no headers.
        names (list, optional): List of column names to use if header is None.
        
    Returns:
        pd.DataFrame: Loaded DataFrame
    """
    csv_path = os.path.join(base_dir, csv_filename)

    try:
        df = pd.read_csv(csv_path, header=header, names=names)
        print(f"✅ Loaded '{csv_filename}' with {len(df)} rows and {len(df.columns)} columns.")
        return df
    except Exception as e:
        print(f"⚠️ Could not load '{csv_filename}': {e}")
        return None


In [78]:
#Use the function load_csv_data to access your 
csv_filename = 'COFires_bats_2024.csv'
csv_df = load_csv_data(csv_filename, data2025_dir)
csv_df.head()

✅ Loaded 'COFires_bats_2024.csv' with 1874 rows and 20 columns.


,site,date,hi_pass,lo_pass,Myyu,Myca,Myci,Myvo,Mylu,Pahe,Anpa,Epfu,Lano,Coto,Myth,Tabr,Laci,Euma,year,area
0,CP1-038,7/22/2024,380,30,0,0,0,16,0,0,0,0,10,0,0,0,0,0,2024,CP1
1,CP1-038,7/23/2024,384,24,0,0,0,11,1,0,0,0,7,0,0,0,0,0,2024,CP1
2,CP1-038,7/24/2024,510,55,0,0,0,15,0,0,0,0,8,0,0,0,0,0,2024,CP1
3,CP1-038,7/25/2024,207,31,0,0,0,4,0,0,0,0,2,0,0,0,0,0,2024,CP1
4,CP1-038,7/26/2024,337,15,0,0,0,3,0,0,0,0,0,0,1,1,1,0,2024,CP1


# Data Wrangling

In [79]:
# Skip the first two rows and load the actual header row (row index 2)
csv_path = "/Users/erinzimmerman/earth-analytics/data2025/wildfire/COFires_bats_2024.csv"
# csv_df = pd.read_csv(csv_path, skiprows=2)

# Preview the cleaned-up DataFrame
csv_df.head()

,site,date,hi_pass,lo_pass,Myyu,Myca,Myci,Myvo,Mylu,Pahe,Anpa,Epfu,Lano,Coto,Myth,Tabr,Laci,Euma,year,area
0,CP1-038,7/22/2024,380,30,0,0,0,16,0,0,0,0,10,0,0,0,0,0,2024,CP1
1,CP1-038,7/23/2024,384,24,0,0,0,11,1,0,0,0,7,0,0,0,0,0,2024,CP1
2,CP1-038,7/24/2024,510,55,0,0,0,15,0,0,0,0,8,0,0,0,0,0,2024,CP1
3,CP1-038,7/25/2024,207,31,0,0,0,4,0,0,0,0,2,0,0,0,0,0,2024,CP1
4,CP1-038,7/26/2024,337,15,0,0,0,3,0,0,0,0,0,0,1,1,1,0,2024,CP1


In [80]:
# Trim whitespace in string colums, just in case
csv_df['site'] = csv_df['site'].str.strip()
csv_df['area'] = csv_df['area'].str.strip()

# Check date types
csv_df.dtypes


site       object
date       object
hi_pass     int64
lo_pass     int64
Myyu        int64
Myca        int64
Myci        int64
Myvo        int64
Mylu        int64
Pahe        int64
Anpa        int64
Epfu        int64
Lano        int64
Coto        int64
Myth        int64
Tabr        int64
Laci        int64
Euma        int64
year        int64
area       object
dtype: object

In [81]:
# Convert dates from being objects to intigers. 
csv_df['date'] = pd.to_datetime(csv_df['date'], errors='coerce')



In [82]:
# Identify missing values
csv_df.isnull().sum()


site       0
date       0
hi_pass    0
lo_pass    0
Myyu       0
Myca       0
Myci       0
Myvo       0
Mylu       0
Pahe       0
Anpa       0
Epfu       0
Lano       0
Coto       0
Myth       0
Tabr       0
Laci       0
Euma       0
year       0
area       0
dtype: int64

In [83]:
# Narrow it down to the correct fire
### figure out which rows are part of Mullen Fire by looking for MU in the site nae
mullen_csv_df = csv_df[csv_df['site'].str.contains("MU", na=False)]

mullen_csv_df


,site,date,hi_pass,lo_pass,Myyu,Myca,Myci,Myvo,Mylu,Pahe,Anpa,Epfu,Lano,Coto,Myth,Tabr,Laci,Euma,year,area
176,SMU14440,2024-05-22,6,5,0,0,0,0,0,0,0,0,2,0,0,0,1,0,2024,SMU
177,SMU14440,2024-05-23,0,4,0,0,0,0,0,0,0,0,0,0,0,0,2,0,2024,SMU
178,SMU14440,2024-05-24,15,7,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2024,SMU
179,SMU14440,2024-05-25,3,3,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2024,SMU
180,SMU14440,2024-05-26,12,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2024,SMU
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1869,MU1-127,2024-08-14,66,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2024,MU1
1870,MU1-127,2024-08-15,11,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2024,MU1
1871,MU1-127,2024-08-16,96,5,0,0,0,0,0,0,0,0,2,0,0,0,0,0,2024,MU1
1872,MU1-127,2024-08-17,38,6,0,0,0,0,0,0,0,0,3,0,0,0,0,0,2024,MU1


In [84]:
# Filter to just 'MU' fire sites
mu_mask = csv_df['site'].str.startswith("MU", na=False)
mullen_csv_df = csv_df[mu_mask].copy()

# Extract severity and site number from 'MUx-yyy' format
pattern = r"MU(\d)-(\d{3})"
mullen_csv_df[['severity', 'site_num']] = mullen_csv_df['site'].str.extract(pattern)

# Drop rows where extraction failed (i.e., the format didn't match)
mullen_csv_df.dropna(subset=['severity', 'site_num'], inplace=True)

# Convert types
mullen_csv_df['severity'] = mullen_csv_df['severity'].astype(int)
mullen_csv_df['site_num'] = mullen_csv_df['site_num'].astype(int)

mullen_csv_df



,site,date,hi_pass,lo_pass,Myyu,Myca,Myci,Myvo,Mylu,Pahe,...,Lano,Coto,Myth,Tabr,Laci,Euma,year,area,severity,site_num
627,MU2-038,2024-07-22,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,2024,MU2,2,38
628,MU2-038,2024-07-23,0,4,0,0,0,0,0,0,...,0,0,0,1,0,0,2024,MU2,2,38
1266,MU4-128,2024-06-05,2,3,0,0,0,0,0,0,...,0,0,0,0,0,0,2024,MU4,4,128
1267,MU4-128,2024-06-06,2,5,0,0,0,0,0,0,...,0,0,0,0,1,0,2024,MU4,4,128
1268,MU4-128,2024-06-07,1,2,0,0,0,0,0,0,...,1,0,0,0,1,0,2024,MU4,4,128
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1869,MU1-127,2024-08-14,66,6,0,0,0,0,0,0,...,0,0,0,0,0,0,2024,MU1,1,127
1870,MU1-127,2024-08-15,11,2,0,0,0,0,0,0,...,0,0,0,0,0,0,2024,MU1,1,127
1871,MU1-127,2024-08-16,96,5,0,0,0,0,0,0,...,2,0,0,0,0,0,2024,MU1,1,127
1872,MU1-127,2024-08-17,38,6,0,0,0,0,0,0,...,3,0,0,0,0,0,2024,MU1,1,127


# Wrangling for the site data. 
This data does not have column headers and is in a different format from the previous data. From this sheet will will need to get the geometry data so that it can be merged with the bat count data. 

In [87]:
# #Use the function load_csv_data to access your 
# csv_filename = 'site_data_2024.csv'
# headers = ['fire_site', 'site', 'latitude', 'longitude', 'date_start', 'date_end','space' ,'space','space','space','space','space','space','space','space','space','space','space','space','space','space','space','space','space','space','space']  # Whatever your structure is

# site_df = load_csv_data(csv_filename, data2025_dir, header=None, names=headers)
# site_df.head()

In [97]:
#Use the function load_csv_data to access your 
csv_filename = 'site_data_2024.csv'
site_df = load_csv_data(csv_filename, data2025_dir)
site_df.head()

✅ Loaded 'site_data_2024.csv' with 134 rows and 26 columns.


,Site,Point Number,Zone,Lattitude,Longitude,Date Set,Date Pulled,TN,Set Type,Ult Mic AZ,...,Detection Angle,Green BA,Red BA,Grey BA,Shrub Cover,Total VO,Total Canopy Cover,Majority Species,Unnamed: 24,Unnamed: 25
0,CP-007,007,CP,40.53952,-105.86966,2024-07-01,2024-07-17,16d,Both,122.0,...,53,9.0,0.0,7,25.0,NaN,NaN,NaN,NaN,NaN
1,CP-008,008,CP,40.58356,-106.00157,2024-07-03,2024-07-22,18d 23h 5m 43s,Both,284.0,...,44,2.0,0.0,1,20.0,NaN,NaN,NaN,NaN,NaN
2,CP-031,031,CP,40.62012,-105.83293,2024-06-11,2024-07-02,21d 4h 39m 47s,Both,52.0,...,36,0.0,0.0,9,1.0,NaN,NaN,NaN,NaN,NaN
3,CP-038,038,CP,40.40217,-105.63556,2024-07-22,2024-08-05,13d 21h 40m 48s,ARU Only,169.0,...,0,16.0,0.0,3,10.0,NaN,NaN,NaN,NaN,NaN
4,CP-045,045,CP,40.68795,-105.86034,2024-06-12,2024-07-01,19d,Both,33.0,...,38,6.0,0.0,8,25.0,NaN,NaN,NaN,NaN,NaN


In [102]:
# Filter to just 'MU' fire sites
mu_mask = site_df['Site'].str.startswith("MU", na=False)
mullen_site_df = csv_df[mu_mask].copy()

mullen_site_df

/var/folders/j7/815c6qns6g39vvr0gw7cftfm0000gn/T/ipykernel_60640/3832127665.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  mullen_site_df = csv_df[mu_mask].copy()


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

In [99]:
# Create a geometry column from lon/lat
geometry = [Point(xy) for xy in zip(site_df['Longitude'], site_df['Lattitude'])]

# Convert to GeoDataFrame
site_gdf = gpd.GeoDataFrame(site_df, geometry=geometry)

# Set the coordinate reference system (CRS) to WGS84 (lat/lon)
site_gdf.set_crs(epsg=4326, inplace=True)


,Site,Point Number,Zone,Lattitude,Longitude,Date Set,Date Pulled,TN,Set Type,Ult Mic AZ,...,Green BA,Red BA,Grey BA,Shrub Cover,Total VO,Total Canopy Cover,Majority Species,Unnamed: 24,Unnamed: 25,geometry
0,CP-007,007,CP,40.53952,-105.86966,2024-07-01,2024-07-17,16d,Both,122.0,...,9.0,0.0,7,25.0,NaN,NaN,NaN,NaN,NaN,POINT (-105.86966 40.53952)
1,CP-008,008,CP,40.58356,-106.00157,2024-07-03,2024-07-22,18d 23h 5m 43s,Both,284.0,...,2.0,0.0,1,20.0,NaN,NaN,NaN,NaN,NaN,POINT (-106.00157 40.58356)
2,CP-031,031,CP,40.62012,-105.83293,2024-06-11,2024-07-02,21d 4h 39m 47s,Both,52.0,...,0.0,0.0,9,1.0,NaN,NaN,NaN,NaN,NaN,POINT (-105.83293 40.62012)
3,CP-038,038,CP,40.40217,-105.63556,2024-07-22,2024-08-05,13d 21h 40m 48s,ARU Only,169.0,...,16.0,0.0,3,10.0,NaN,NaN,NaN,NaN,NaN,POINT (-105.63556 40.40217)
4,CP-045,045,CP,40.68795,-105.86034,2024-06-12,2024-07-01,19d,Both,33.0,...,6.0,0.0,8,25.0,NaN,NaN,NaN,NaN,NaN,POINT (-105.86034 40.68795)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,MU-91,91,MU,41.19696,-106.30634,2024-06-18,2024-07-09,21d,Both,41.0,...,8.0,0.0,1,5.0,NaN,NaN,NaN,NaN,NaN,POINT (-106.30634 41.19696)
130,Sf-102,102,Sf,40.53227,-105.95099,2024-07-24,2024-08-13,20d,Both,355.0,...,8.0,0.0,0,5.0,NaN,NaN,NaN,NaN,NaN,POINT (-105.95099 40.53227)
131,Sf-151,151,Sf,40.54405,-105.93614,2024-07-24,2024-08-13,20d,Both,251.0,...,12.0,0.0,0,2.0,NaN,NaN,NaN,NaN,NaN,POINT (-105.93614 40.54405)
132,Sf-43,43,Sf,40.54015,-104.95400,2024-07-24,2024-08-13,20d 1h 33m 11s,Both,321.0,...,7.0,0.0,0,5.0,NaN,NaN,NaN,NaN,NaN,POINT (-104.954 40.54015)


In [ ]:
# Preview the site on the map
site_gdf.explore()


In [ ]:
# Store the essential information to import into the next notebook
%store 
